# Naerk: Data Analysis

## Table of Contents
- [Deities](#deities)
    - [Deity Characteristics](#deity_characteristics)
    - [Devotee Benefits](#devotee_benefits)
    - [Stats: frequencies](#stats:_frequencies)
        - [Category](#category)
        - [Portfolio](#portfolio)
        - [Followers](#followers)
        - [Pantheon](#pantheon)
        - [Plane](#plane)
        - [Divine Ability](#divine_ability)
        - [Divine Ability](#divine_ability_0)
        - [Divine Skill](#divine_skill)
        - [Favored Weapon](#favored_weapon)
        - [Domains](#domains)
- [Countries](#countries)
    - [Stats: frequencies](#stats:_frequencies_0_0)
        - [Climate](#climate)
        - [Government](#government)
        - [Languages](#languages)
        - [Religions](#religions)
        - [Imports](#imports)
        - [Exports](#exports)
- [Factions](#factions)
- [Planes](#planes)
    - [Stats: frequencies](#stats:_frequencies_0)
        - [Traits](#traits)
        - [Category](#category_0)
        - [Divinities](#divinities)
        - [Native Inhabitants](#native_inhabitants)

In [42]:
import os
import re
from datetime import datetime

print("v.", datetime.now())

# Specify the directory where markdown files are located
content_dir = '/home/ulysses/Documents/Naerk/naerk-wiki/quartz/content'

def build_tsv_data(md_category):
    """
    generates tab-delimited files from markdown files in a directory
    assumes that the data is contained in tables that are within callouts
        with the expectation that:
        TABLES WITHIN CALLOUTS SHOULD ONLY OCCUR IN WIKITABLES
    md_category: the markdown directory
    """

    category_dir = f"{content_dir}/{md_category}"

    csv_dir = f"{category_dir}/.data"
    if not os.path.exists(csv_dir): 
        os.makedirs(csv_dir)

    for filename in os.listdir(category_dir):
        if filename.endswith('.md'):
            file_path = os.path.join(category_dir, filename)

            with open(file_path,'r') as input:
                # read tables within callouts
                targets = [line for line in input if "> > | " in line]
            name = filename.replace(".md","")
            delimiter = '\t'
            with open(f"{csv_dir}/{name}.tsv", 'w') as output:
                output.write(f"Name\t{name}\n")
                for line in targets:
                    parsed_line = line
                    # parsed_line = parsed_line.replace("\"","'")

                    # parse callout and table syntax
                    parsed_line = parsed_line.replace("> > | ","")
                    parsed_line = parsed_line.replace(" | ","\t")
                    parsed_line = parsed_line.replace(" |","")
                    
                    # remove link syntax (keep target name if renamed)
                    parsed_line = re.sub(r"\[\[(.*?)\|", "", parsed_line)
                    parsed_line = parsed_line.replace("]]","")
                    parsed_line = parsed_line.replace("[[","")
                    parsed_line = parsed_line.replace("|\n","")
                    # remove table subheader row md syntax
                    parsed_line = parsed_line.replace("---\t---\n","")
                    # EXPECTATION: 'or' separate list items
                    parsed_line = parsed_line.replace(" or ",", ")
                    #  remove double quotes
                    parsed_line = parsed_line.replace('"', "")
                    # EXPECTATION: '-' denotes NaN values
                    parsed_line = parsed_line.replace("\t-","\t")

                    output.write(parsed_line)

build_tsv_data("Countries")
build_tsv_data("Gods")
build_tsv_data("Factions")
build_tsv_data("Planes")

v. 2024-03-15 19:42:42.037526


In [43]:
import glob
import pandas as pd
from IPython.display import display, Markdown

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)


def build_dataframe(md_category):

    category_dir = f"{content_dir}/{md_category}"
    data_directory = f"{category_dir}/.data"

    os.chdir(data_directory)
    extension = 'tsv'
    all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

    dataframes = []  # List to store the DataFrames

    for csv_file in all_filenames:
        df = pd.read_csv(csv_file, sep='\t')
        dataframes.append(df)

    merged_df = pd.merge(dataframes[0], dataframes[1], on='Name', how='left', sort=False)

    for i in range(2, len(dataframes)):
        merged_df = pd.merge(merged_df, dataframes[i], on='Name', how='left', sort=False)

    # older implementation:
    # df = merged_df.set_index('Name').transpose().sort_index(ascending=True)
        
    df = merged_df.T.reset_index()
    df.columns = df.loc[0]
    df = df.drop(0).sort_values(by="Name").reset_index(drop=True)
    
    return(df)


def filter_dataframe(df, cols):
    """
    df:      dataframe created by build_dataframe function
    cols: a list containing names of columns to be filtered
    """

    dff = df.set_index('Name').rename_axis(None).filter(items=cols)

    return(dff)


def df_freq(df, col, xls=False):
    """
    df:  the master dataframe from build_dataframe
    col: field to be analysed; a column in df
    xls: set to True to produce analytical excel file
    """

    if xls == True:
        excelname = f'{md_category}.xlsx'
        df.to_excel(excelname, sheet_name="Data", index=False)
        # df.to_csv("_dataframe.csv")

    # col column: strings = values delimited by commas; turn them to list
    dfy = df.copy()
    dfy[col] = dfy[col].str.split(', ')

    # explode the lists in the col column
    dfx = dfy.explode(col)

    # get value counts of unique values in the lists
    xvaluecounts = dfx[col].value_counts().reset_index()

    # get a list of 'Name' values for each unique value in the lists
    name_list = dfx.groupby(col)['Name'].agg(list).reset_index()

    # merge the two dataframes on the col column
    dfm = pd.merge(xvaluecounts, name_list, on=col)

    # make pretty for output
    dfm['Name'] = dfm['Name'].str.join(', ')
    dfm = dfm.rename(columns={'Name': 'occurences'})
    dfm = dfm.set_index(col).rename_axis(None)

    # display
    display(dfm)

    if xls == True:
        with pd.ExcelWriter(excelname, engine='openpyxl', mode='a') as writer:
            dfx.to_excel(writer, sheet_name=col, index=False)

## Deities <a class="anchor" id="deities"></a>


In [44]:
md_category = "Gods"
df = build_dataframe(md_category)

### Deity Characteristics <a class="anchor" id="deity_characteristics"></a>


In [45]:
columns = ['Name', 'Title', 'Category', 'Edicts', 'Portfolio', 'Anathema', 'Followers', 'Pantheon', 'Plane']

display(filter_dataframe(df, columns))

,Title,Category,Edicts,Portfolio,Anathema,Followers,Pantheon,Plane
Amehan,Destroyer,NaN,"Destroy material plane, Loss","Destruction, Razing Structures, Pillaging, Civilization's End, Despair",Creating beautiful things,"Warlords, pillagers, invaders, nihilists, savages, goblinoids, titans, giants, Vulga, Erevi",NaN,Destruction
Anhalt,Unrelenting,Ascended,"Unstoppable Will, Sacrifice, Duty to the end","Sacrifice, Honor, Valor, Martyrdom","Ignore duty, orders, Show fear","Warriors, bodyguards, knights, defenders, dwarves, Hurtsvigsen",Dwarven,"Kelzadam, Fortress of the Just"
Auromia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B'tolen,Raven's Flight,Aspects,"Meet new civilizations, constant learning, wondering and wandering","Unbound travel, Learning, Luck, Exploration",Permanent settlers,"travellers, wanderers, pilgrims, sailors, elves, tribes","Tribal, Elven, Druidic",Evergrowing Forest
Berohath,Fox's Cunning,Aspects,"Lie and Deceive fools, Take advantage of situations, Spread doubt","Stealth, Thievery, Malice, Luck, Night","Announce intentions, Give back stolen things","travellers, thieves, bagabonds, beggars, downtrodden, scavengers, rogues, elves, tribes","Tribal, Elven, Goblin",Evergrowing Forest
Crel'vach,The Bear's Fury,Aspects,"Fury of the Wild, Berserker Rage, Unrelenting Combat","Berserkers, warriors, endurance, savagery","Not answering an insult, cowardice","Berserkers, warriors, vanguards, gladiators, orcs, Hurtsvigsen","Tribal, Orc",Evergrowing Forest
Disiklek,The Plague Lord,"Archdemon, Chosen","Spread Disease, Plague and Poisons, Spoil communities and food, Wait to take its course","Disease, Plague, Poison, Cannibalism, Virulent Fungi and Vermin","Stop a plague, cure a disease, poison","Poisoners, Alchemists, Lunatics, Serpentfolk Crimson Steel","Serpentfolk, Demonic",Abyss
Drokasner,Thunderstrike,Aspects,"Bad Weather, Fury of the Sky, Defiance","Thunder, Storm, Bad Temper","Try to calm, avoid weather, Disrespect the elements","travellers, mountaineers, sailors, orcs, giants, Hoen","Tribal, Orc, Giant","Elemental Ring, Evergrowing Forest"
Equalizer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Factar,The Compassionate,"Luminous Walkers, Human","Help those in need, Humility, Aid by example","Healing, Farming, Fertility, Wanderers, Poor","Αrrogance, Egotism above others, Cold shoulder","Hospitalers, Healers, Pilgrims, travellers, Midwives, Herbalists, Priests, Commonfolk, Luminous Order","Luminous Walkers, halfling",Golden Palace


### Devotee Benefits <a class="anchor" id="devotee_benefits"></a>


In [46]:
columns = ['Name', 'Divine Ability', 'Divine Font', 'Divine Skill', 'Favored Weapon', 'Domains', 'Cleric Spells']

display(filter_dataframe(df, columns))


,Divine Ability,Divine Font,Divine Skill,Favored Weapon,Domains,Cleric Spells
Amehan,"Strength, Dexterity",Harm,Athletics,Mace,"Destruction, Decay, Dust, Pain, Sorrow, Void","1st: Phantom Pain, 6th: Disintegrate, 9th: Disjunction"
Anhalt,"Constitution, Wisdom","Heal, Harm",Athletics,Warhammer,"Confidence, Duty, Fire, Might, Protection, Vigil, Zeal","1st: True Strike, 4th: Fire Shield, 5th: Mantle of the Magma Heart"
Auromia,NaN,NaN,NaN,NaN,NaN,NaN
B'tolen,"Dexterity, Intelligence","Heal, Harm",Society,Longbow,"Air, Change, Freedom, Knowledge, Swarm, Travel","1st: Animal Allies, 2nd: Animal Form, 9th: Shapechange"
Berohath,"Dexterity, Intelligence",Harm,Stealth,Dagger,"Darkness, Luck, Moon, Secrecy, Trickery","1st: Invisible Item, 2nd: Invisibility, 5th: Chameleon Coat"
Crel'vach,"Strength, Constitution","Heal, Harm",Intimidate,Greataxe,"Confidence, Might, Nature, Protection, Zeal","1st: Magic Fang, 2nd: Enlarge, 4th: Weapon Storm"
Disiklek,"Intelligence, Constitution",Harm,Crafting,Scythe,"Decay, Delirium, Pain, Plague, Sorrow","1st: Noxious Vapors, 3rd: Gasping Marsh, 5th: Cloudkill"
Drokasner,"Constitution, Wisdom",Harm,Survival,Spear,"Air, Confidence, Lightning, Might","1st: Horizon Thunder Sphere, 5th: Stormburst, 6th: Chain Lightning"
Equalizer,NaN,NaN,NaN,NaN,NaN,NaN
Factar,"Wisdom, Charisma",Heal,Medicine,Sickle,"Cities, Family, Healing, Protection, Sun, Travel, Wood","1st: Soothe, 3rd: Cozy Cabin, 6th: Collective Transposition"


### Stats: frequencies <a class="anchor" id="stats:_frequencies"></a>


#### Category <a class="anchor" id="category"></a>


In [73]:
df_freq(df, 'Category')

,count,occurences
Elemental Ring,6,"Air Quarter, Earth Quarter, Elemental Ring, Fire Quarter, Kelzadam, Water Quarter"
Outer,6,"Creation, Destruction, Fortress of the Just, Golden Palace, Hell, Zinctizuctra"
Fifth Dimension,6,"Dreamworld, Ethereal Plane, Evergrowing Forest, Shadow Plane, Utopia, Void"
River Styx,4,"Abyss, Tailway, Tartara, Zaramel"
Elemental,1,Material Plane
Demiplane,1,The Infinite Library


#### Portfolio <a class="anchor" id="portfolio"></a>


In [48]:
df_freq(df, 'Portfolio')

,count,occurences
Life,3,"Malahir, Nahema, Reszek"
Vigilance,3,"Lleyn, Tarov, Yerastu"
Community,3,"Lastra, Mormankar, Vard"
Destruction,2,"Amehan, Yast"
Youth,2,"Hedbba, Valanshore"
Liberation,2,"Fyrlean, Lainek"
Earth,2,"Gibba, Tarov"
Stone,2,"Gibba, Tarov"
Books,2,"Hansa, Yushan"
Writing,2,"Hansa, Yushan"


#### Followers <a class="anchor" id="followers"></a>


In [49]:
df_freq(df, 'Followers')

,count,occurences
tribes,9,"B'tolen, Berohath, Jalerk, Lastra, Mormankar, Sseratush, Tarov, Valanshore, Volrek"
travellers,8,"B'tolen, Berohath, Drokasner, Factar, Hedbba, Hydarax, Maeronian, Saekla"
elves,7,"B'tolen, Berohath, Fallfaunt, Lastra, Malahir, Valanshore, Volrek"
dwarves,6,"Anhalt, Gibba, Klont, Kromlek, Tarov, Vard"
gnomes,6,"Hedbba, Klont, Malahir, Mormankar, Tarov, Volrek"
sailors,5,"B'tolen, Drokasner, Jalerk, Maeronian, Saekla"
acolytes,4,"Fyrlean, Il'vec, Tarov, Voolthow"
giants,4,"Amehan, Drokasner, Fallfaunt, Gibba"
engineers,3,"Gibba, Mormankar, Vard"
pilgrims,3,"B'tolen, Hansa, Saekla"


#### Pantheon <a class="anchor" id="pantheon"></a>


In [50]:
df_freq(df, 'Pantheon')

,count,occurences
Tribal,12,"B'tolen, Berohath, Crel'vach, Drokasner, Fallfaunt, Il'vec, Jalerk, Lastra, Mormankar, Sseratush, Tarov, Valanshore"
Elven,8,"B'tolen, Berohath, Fallfaunt, Huntsman, Lastra, Malahir, Valanshore, Volrek"
Dwarven,5,"Anhalt, Gibba, Klont, Kromlek, Vard"
Druidic,5,"B'tolen, Il'vec, Lastra, Malahir, Volrek"
Elemental,5,"Fallfaunt, Fyrlean, Hydarax, Jalerk, Saekla"
Luminous Walkers,4,"Factar, Laerion, Malahir, Reszek"
Gnome,4,"Hedbba, Klont, Mormankar, Tarov"
Demonic,3,"Disiklek, Otynhash, Xeurim"
Divinalis,3,"Hansa, Sigard, Sumril"
Giant,3,"Drokasner, Fallfaunt, Tarov"


#### Plane <a class="anchor" id="plane"></a>


In [51]:
df_freq(df, 'Plane')

,count,occurences
Evergrowing Forest,16,"B'tolen, Berohath, Crel'vach, Drokasner, Fallfaunt, Huntsman, Il'vec, Jalerk, Karadosh, Lastra, Malahir, Mormankar, Sseratush, Tarov, Valanshore, Volrek"
Elemental Ring,10,"Drokasner, Fallfaunt, Fyrlean, Gibba, Hydarax, Jalerk, Klont, Saekla, Tarov, Vard"
Fortress of the Just,5,"Anhalt, Laerion, Lleyn, Nentha, Yerastu"
Kelzadam,4,"Anhalt, Klont, Kromlek, Vard"
Abyss,3,"Disiklek, Otynhash, Xeurim"
Golden Palace,3,"Factar, Reszek, Sigard"
Utopia,3,"Hansa, Sennek, Sumril"
Zinctizuctra,3,"Maeronian, Steenar, Yushan"
Destruction,2,"Amehan, Neverborn"
Zaramel,2,"Hedbba, Lainek"


#### Divine Ability <a class="anchor" id="divine_ability"></a>


In [52]:
df_freq(df, 'Pantheon')

,count,occurences
Tribal,12,"B'tolen, Berohath, Crel'vach, Drokasner, Fallfaunt, Il'vec, Jalerk, Lastra, Mormankar, Sseratush, Tarov, Valanshore"
Elven,8,"B'tolen, Berohath, Fallfaunt, Huntsman, Lastra, Malahir, Valanshore, Volrek"
Dwarven,5,"Anhalt, Gibba, Klont, Kromlek, Vard"
Druidic,5,"B'tolen, Il'vec, Lastra, Malahir, Volrek"
Elemental,5,"Fallfaunt, Fyrlean, Hydarax, Jalerk, Saekla"
Luminous Walkers,4,"Factar, Laerion, Malahir, Reszek"
Gnome,4,"Hedbba, Klont, Mormankar, Tarov"
Demonic,3,"Disiklek, Otynhash, Xeurim"
Divinalis,3,"Hansa, Sigard, Sumril"
Giant,3,"Drokasner, Fallfaunt, Tarov"


#### Divine Ability <a class="anchor" id="divine_ability_0"></a>


In [53]:
df_freq(df, 'Divine Ability')

,count,occurences
Dexterity,18,"Amehan, B'tolen, Berohath, Fallfaunt, Fyrlean, Hedbba, Huntsman, Hydarax, Il'vec, Karadosh, Lainek, Maeronian, Nahema, Saekla, Sseratush, Steenar, Valanshore, Xeurim"
Constitution,18,"Anhalt, Crel'vach, Disiklek, Drokasner, Gibba, Il'vec, Klont, Kromlek, Lleyn, Malahir, Mormankar, Neverborn, Otynhash, Sumril, Tarov, Vard, Veiroch, Volrek"
Intelligence,17,"B'tolen, Berohath, Disiklek, Hansa, Lainek, Lastra, Maeronian, Mormankar, Nahema, Saekla, Sennek, Sigard, Steenar, Vard, Yast, Yerastu, Yushan"
Charisma,17,"Factar, Fallfaunt, Fyrlean, Hedbba, Jalerk, Klemnestron, Laerion, Lleyn, Nentha, Sennek, Sergonial, Sigard, Sseratush, Veiroch, Voolthow, Xeurim, Yast"
Wisdom,16,"Anhalt, Drokasner, Factar, Hansa, Huntsman, Hydarax, Jalerk, Lastra, Malahir, Nentha, Reszek, Sumril, Volrek, Voolthow, Yerastu, Yushan"
Strength,14,"Amehan, Crel'vach, Gibba, Karadosh, Klemnestron, Klont, Kromlek, Laerion, Neverborn, Otynhash, Reszek, Sergonial, Tarov, Valanshore"


#### Divine Skill <a class="anchor" id="divine_skill"></a>


In [54]:
df_freq(df, 'Divine Skill')

,count,occurences
Athletics,7,"Amehan, Anhalt, Hydarax, Kromlek, Laerion, Otynhash, Valanshore"
Crafting,6,"Disiklek, Fyrlean, Gibba, Klont, Mormankar, Nahema"
Society,6,"B'tolen, Lleyn, Nentha, Reszek, Tarov, Vard"
Survival,4,"Drokasner, Huntsman, Karadosh, Lastra"
Arcana,4,"Hansa, Maeronian, Sumril, Yushan"
Stealth,3,"Berohath, Sseratush, Steenar"
Occultism,3,"Jalerk, Neverborn, Yast"
Intimidate,2,"Crel'vach, Klemnestron"
Performance,2,"Fallfaunt, Hedbba"
Thievery,2,"Il'vec, Lainek"


#### Favored Weapon <a class="anchor" id="favored_weapon"></a>


In [55]:
df_freq(df, 'Favored Weapon')

,count,occurences
Dagger,3,"Berohath, Steenar, Xeurim"
Greataxe,3,"Crel'vach, Otynhash, Veiroch"
Spear,3,"Drokasner, Malahir, Nahema"
Longsword,3,"Laerion, Nentha, Yerastu"
Jaws,3,"Il'vec, Karadosh, Sseratush"
Bastard Sword,2,"Lleyn, Yast"
Hatchet,2,"Karadosh, Mormankar"
Sickle,2,"Factar, Il'vec"
Morningstar,2,"Fallfaunt, Reszek"
Maul,2,"Gibba, Klont"


#### Domains <a class="anchor" id="domains"></a>


In [56]:
df_freq(df, 'Domains')

,count,occurences
Protection,14,"Anhalt, Crel'vach, Factar, Hansa, Kromlek, Laerion, Lastra, Mormankar, Reszek, Sigard, Tarov, Vard, Volrek, Yerastu"
Might,12,"Anhalt, Crel'vach, Drokasner, Gibba, Karadosh, Klemnestron, Kromlek, Laerion, Otynhash, Sergonial, Sseratush, Valanshore"
Travel,11,"B'tolen, Factar, Hedbba, Huntsman, Hydarax, Lleyn, Maeronian, Saekla, Sennek, Tarov, Yushan"
Cities,10,"Factar, Hansa, Lainek, Mormankar, Nahema, Nentha, Sennek, Sigard, Vard, Xeurim"
Confidence,9,"Anhalt, Crel'vach, Drokasner, Fallfaunt, Fyrlean, Klemnestron, Otynhash, Sseratush, Valanshore"
Duty,9,"Anhalt, Kromlek, Laerion, Lleyn, Nentha, Sergonial, Vard, Veiroch, Yerastu"
Family,9,"Factar, Laerion, Lainek, Lastra, Mormankar, Nahema, Sigard, Sseratush, Vard"
Sorrow,8,"Amehan, Disiklek, Fallfaunt, Jalerk, Klemnestron, Neverborn, Voolthow, Yast"
Zeal,8,"Anhalt, Crel'vach, Fyrlean, Huntsman, Kromlek, Laerion, Reszek, Veiroch"
Nature,8,"Crel'vach, Huntsman, Hydarax, Karadosh, Lastra, Malahir, Saekla, Volrek"


## Countries

In [57]:
md_category = "Countries"
df = build_dataframe(md_category)

In [58]:
columns = ['Name', 'Climate', 'Capital', 'Government', 'Languages', 'Population', 'Religions', 'Imports', 'Exports']

display(filter_dataframe(df, columns))

,Climate,Capital,Government,Languages,Population,Religions,Imports,Exports
Alagon,Tropical Wet-and-Dry,Qillport,Confederate Oligarchy,"Common, Elven, Halfling,",80000,"Jalerk, Yerastu, Sennek, Hydarax, Yushan, Drokasner, Klemnestron",NaN,NaN
Allwick,Arid Desert,NaN,Loose,"Common, Auran, Ignan, Gnoll, Shadow","Unknown, at least 580000","Saekla, Fyrlean, Yushan, Maeronian, Hansa, Sumril, Voolthow, Il'vec","Steel, Timber, Grain, Corn, Fruits","Salt, Magic Items, Alchemical Items, Books, Sculpture"
Arbader,Inner Sea,Hashnev,King,"Common, Goblin, Giant, Shadow",870000,"Sergonial, Xeurim, Sennek, Berohath, Factar","Wool, Oil, Spices, Timber","Gold, Jewelry, Grain, Wine"
Athar,"Temperate Continental, Tundra",NaN,Tribal Council,Orcish,80000,"Malahir, Karadosh, Huntsman, B'tolen, Crel'vach",NaN,NaN
Belgosreim,"Continental Cold, Tundra",Clapten,Tribal Monarchy,"Common, Giant, Elvish, Orcish, Goblin",249000,"Jalerk, Drokasner, Fallfaunt, Crel'vach, Huntsman","Fruit, Silk, Precious Metals, Olive Oil, Beer","Fish, Whale Oil, Timber, Tamed Beasts, Ships, Sailors"
Bjåglant,"Boreal, Tundra",Bärshamn,Clan Monarchy,"Common, Dwarven, Giant, Orcish",190000,"Drokasner, Crel'vach, Volrek, Lastra, Klont, Vard, Kromlek","Fruit, Grain, Silk","Fish, Forged Metal, Precious Metals, Timber"
Caulnorr,"Continental Cold, Taiga",Caulnorr,King,"Giant, Goblin, Gnoll",304000,"Sergonial, Amehan, Berohath, Karadosh, Gibba","Meat, Livestock, Salt, Grain, Slaves","Weapons, Armors, Stonework, Marble, Granite"
Darrok,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Dragonkeep,"Continental Cold, Taiga",Ralzyn,Draconic Monarchy,Draconic,670000,Klemnestron,"Gold, Jewelry, Gems, Artwork, Meat, Tourists","Fish, Forged Metals, Artwork, Rare Minerals"
Ednais,Cold,Ednais,Draconic Oligarchy,Draconic,288000,"Klemnestron, Amehan","Meat, Livestock, Gold, Gems, Pearls, Slaves","Weapons, Timber, Grain, Oats, Potato"


### Stats: frequencies

#### Climate

In [59]:
df_freq(df, 'Climate')

,count,occurences
Temperate,8,"Green Hills, Imtaria, Marelthia, Oromil, Paraselt, Stormfall, Tithlon, Urdankh"
Warm,5,"Hithlwin, Imtaria, Kelonarie, Oromil, Reisdok"
Continental Cold,5,"Belgosreim, Caulnorr, Dragonkeep, Hoen, Korhond"
Inner Sea,4,"Arbader, South Darrok, Sunshine, Zolkar"
Tropical,3,"Ngamui Isles, Whistor, Zelgerosh"
Tundra,3,"Athar, Belgosreim, Bjåglant"
Taiga,3,"Caulnorr, Dragonkeep, Korhond"
Temperate Oceanic,2,"Kwarix, Sennarin"
Cold Continental,2,"Kambah, Oventhart"
Temperate Cold,2,"Olgizmar, Tarthyn"


#### Government

In [60]:
df_freq(df, 'Government')

,count,occurences
King,5,"Arbader, Caulnorr, Green Hills, Hoen, Marelthia"
Tribal Council,3,"Athar, Paraselt, Tithlon"
Loose,3,"Allwick, Hithlwin, Ngamui Isles"
Queen,3,"Kelonarie, Naryal, Vayreef"
Governor,2,"South Darrok, Stormfall"
Tribal Monarchy,2,"Belgosreim, Korhond"
Theocratic Monarchy,2,"Urdankh, Zelgerosh"
Oligarchy,2,"Iazip, Tarthyn"
Council,2,"Reisdok, Zolkar"
Warlord,1,Saemorr


#### Languages

In [61]:
df_freq(df, 'Languages')

,count,occurences
Common,28,"Alagon, Allwick, Arbader, Belgosreim, Bjåglant, Green Hills, Hoss Glacier, Imtaria, Kambah, Karrokh, Korhond, Marelthia, Markesh, Ngamui Isles, Northern Darrok, Olgizmar, Oromil, Oventhart, Reisdok, Sennarin, South Darrok, Stormfall, Sunshine, Tarthyn, Tithlon, Urdankh, Whistor, Zolkar"
Giant,12,"Arbader, Belgosreim, Bjåglant, Caulnorr, Hoen, Hoss Glacier, Kambah, Korhond, Ngamui Isles, Northern Darrok, Paraselt, Thimvert Reach"
Goblin,10,"Arbader, Belgosreim, Caulnorr, Hoen, Hoss Glacier, Kambah, Paraselt, Rammok Wastes, Stormfall, Zolkar"
Elven,10,"Alagon, Kelonarie, Naryal, Oventhart, Sennarin, South Darrok, Stormfall, Tarthyn, Tithlon, Zolkar"
Orcish,8,"Athar, Belgosreim, Bjåglant, Hoen, Hoss Glacier, Korhond, Northern Darrok, Olgizmar"
Draconic,6,"Dragonkeep, Ednais, Karrokh, Paraselt, Sunshine, Zelgerosh"
Gnoll,5,"Allwick, Caulnorr, Karrokh, Rammok Wastes, Zolkar"
Halfling,5,"Alagon, Green Hills, Marelthia, Oromil, Tarthyn"
Elvish,4,"Belgosreim, Hoss Glacier, Korhond, Ngamui Isles"
Lizardtongue,4,"Ngamui Isles, Reisdok, Whistor, Zelgerosh"


#### Religions

In [62]:
df_freq(df, 'Religions')

,count,occurences
Reszek,9,"Green Hills, Ngamui Isles, Oromil, Oventhart, Sennarin, South Darrok, Sunshine, Tarthyn, Zelgerosh"
Yerastu,9,"Alagon, Green Hills, Hithlwin, Imtaria, Marelthia, Sennarin, Stormfall, Sunshine, Zolkar"
Amehan,8,"Caulnorr, Ednais, Hoen, Hoss Glacier, Paraselt, Rammok Wastes, Saemorr, Stormfall"
Drokasner,8,"Alagon, Belgosreim, Bjåglant, Hoen, Hoss Glacier, Kambah, Saemorr, Thimvert Reach"
Factar,8,"Arbader, Green Hills, Imtaria, Marelthia, Oromil, Oventhart, South Darrok, Tarthyn"
Hansa,7,"Allwick, Hithlwin, Iazip, Karrokh, Markesh, Sennarin, Whistor"
Karadosh,7,"Athar, Caulnorr, Karrokh, Oromil, Paraselt, Rammok Wastes, Zolkar"
Malahir,7,"Athar, Imtaria, Kelonarie, Marelthia, Naryal, Tarthyn, Urdankh"
Valanshore,7,"Kelonarie, Korhond, Naryal, Oromil, Oventhart, Tarthyn, Tithlon"
Jalerk,7,"Alagon, Belgosreim, Hoss Glacier, Thimvert Reach, Urdankh, Vayreef, Whistor"


#### Imports

In [63]:
df_freq(df, 'Imports')

,count,occurences
Grain,10,"Allwick, Bjåglant, Caulnorr, Hoen, Korhond, Northern Darrok, Rammok Wastes, Reisdok, Sunshine, Zolkar"
Timber,8,"Allwick, Arbader, Marelthia, Markesh, Oromil, Paraselt, South Darrok, Whistor"
Gold,8,"Dragonkeep, Ednais, Hithlwin, Oventhart, Paraselt, Sunshine, Tarthyn, Urdankh"
Meat,8,"Caulnorr, Dragonkeep, Ednais, Iazip, Olgizmar, Thimvert Reach, Tithlon, Urdankh"
Beer,7,"Belgosreim, Hoen, Korhond, Marelthia, Markesh, Tithlon, Whistor"
Slaves,6,"Caulnorr, Ednais, Hithlwin, Saemorr, Stormfall, Vayreef"
Livestock,6,"Caulnorr, Ednais, Hithlwin, Hoen, Hoss Glacier, Sennarin"
Salt,6,"Caulnorr, Green Hills, Naryal, Oventhart, Sunshine, Tarthyn"
Corn,6,"Allwick, Iazip, Rammok Wastes, Stormfall, Sunshine, Zolkar"
Fruit,5,"Belgosreim, Bjåglant, Korhond, Olgizmar, Sennarin"


#### Exports

In [64]:
df_freq(df, 'Exports')

,count,occurences
Fish,16,"Belgosreim, Bjåglant, Dragonkeep, Hoss Glacier, Iazip, Korhond, Ngamui Isles, Olgizmar, Reisdok, Sunshine, Thimvert Reach, Tithlon, Urdankh, Vayreef, Whistor, Zolkar"
Slaves,11,"Hithlwin, Hoen, Kambah, Markesh, Olgizmar, Rammok Wastes, Saemorr, Stormfall, Vayreef, Whistor, Zelgerosh"
Timber,10,"Belgosreim, Bjåglant, Ednais, Green Hills, Imtaria, Kambah, Korhond, Northern Darrok, Stormfall, Tarthyn"
Grain,9,"Ednais, Green Hills, Hithlwin, Imtaria, Marelthia, Oromil, Oventhart, Sennarin, South Darrok"
Livestock,8,"Green Hills, Imtaria, Karrokh, Marelthia, Oromil, Oventhart, South Darrok, Zolkar"
Gems,7,"Hoss Glacier, Markesh, Ngamui Isles, Northern Darrok, Olgizmar, Paraselt, Sunshine"
Magic Items,7,"Allwick, Kelonarie, Markesh, Naryal, Olgizmar, Reisdok, South Darrok"
Artwork,7,"Dragonkeep, Iazip, Markesh, Oventhart, Sunshine, Tithlon, Whistor"
Books,6,"Allwick, Markesh, Oventhart, Sunshine, Tarthyn, Whistor"
Sailors,6,"Belgosreim, Korhond, Stormfall, Thimvert Reach, Urdankh, Vayreef"


## Factions

In [65]:
md_category = "Factions"
df = build_dataframe(md_category)

In [66]:
columns = ['Name', 'Meaning', 'Symbol', 'Region', 'Capital', 'Leader']
display(filter_dataframe(df, columns))

,Meaning,Symbol,Region,Capital,Leader
Arlas,Three Wolves,Three Wolf heads,South Fervolosh,Ferglund,Galnagh Tirl Daralosh
Aspects,NaN,NaN,NaN,NaN,NaN
Baldosh,Wise Dragon,Knotted dragon,"South Foltar, South Volostrol, Norfol",Tarfuhl,Jarl Carlog Gorberosh
Berenfaer,NaN,pegasi,"Zolkar, Karrokh",Elestair,"numerous, including Steelshade and Black Mist"
Bloodscale,NaN,Fire between dragon wings,"Paraselt, Drozzin, Scalemist, Saltwall, western Tarhelm many other frontier areas",NaN,several dragons and sorcerer wyrmscale
Bur'gg,Lakemen,Two serpents forming a lake,"Åperbung, east Volostrol, north Volosherg",Bur'gg,Jarlskona Darma
Crel,Bear,Bearpaw with two axeheads,Southwest Fervolosh,Kreida,Jarl Snåfr Kugroên
Dal'Vorag,Broken Skull,Broken skull,"East Fervolosh, West Foltar",No'Crel,Elder Gorhald Baragon
Daral,Fir,Shapable tree branch with leaf,Northwest Fervolosh,Volanir,Queen Alinwe
Defeated Army,NaN,A cross of four knives,"Paraselt, west Tithlon, southeast Green Hills",NaN,NaN


## Planes <a class="anchor" id="planes"></a>


In [67]:
md_category = "Planes"
df = build_dataframe(md_category)

In [68]:
columns = ['Name', 'Traits', 'Category', 'Divinities', 'Native Inhabitants']
display(filter_dataframe(df, columns))

,Traits,Category,Divinities,Native Inhabitants
Abyss,NaN,River Styx,"Xeurim, Otynhash, Diisiklek, Fiendish Lords","Demons, tortured souls, stolen souls, other fiends, cultist souls"
Air Quarter,"Air Essence, Microgravity, Enhanced Magic (Air)",Elemental Ring,"Saekla, Drokasner","Air Elementals, Sky Dragons, winged creatures, windfolk souls"
Creation,"Timeless, Sentient Morphic, Positive Essence",Outer,Nahema,"Positive Outsiders, devoted souls"
Destruction,"Timeless, Sentient Morphic, Negative Essence",Outer,"Amehan, Neverborn","Negative Outsiders, devoted souls"
Dreamworld,"Immeasurable Scope, Microgravity, Erratic Time, Enhanced Magic (Occult)",Fifth Dimension,NaN,"dreams, nightmares, occult dragons, abominations, sleeping souls"
Earth Quarter,"Earth Essence, High Gravity, Enhanced Magic (Earth)",Elemental Ring,"Gibba, Tarov","Earth Elementals, Gem Dragons"
Elemental Ring,Enhanced Magic (Primal),Elemental Ring,"Gibba, Fyrlean, Saekla, Hydarax",Elementals
Ethereal Plane,"Immeasurable scope, Subjective Gravity, Timeless, Enhanced Magic (Occult)",Fifth Dimension,NaN,"spectres, phantoms, ghosts, occult dragons, wandering souls"
Evergrowing Forest,"Immeasurable Scope, Erratic Time, Sentient Morphic, Enhanced Magic (Primal, Positive)",Fifth Dimension,"Malahir, Aspects","agathions, animalistic outsiders, fey, animal souls, plant souls, tribal souls, savage souls, devoted souls"
Fire Quarter,"Fire Essence, Enhanced Magic (Fire)",Elemental Ring,"Fyrlean, Klemnestron","Fire elementals, red dragons, fire giants, devoted souls, firefolk souls"


### Stats: frequencies <a class="anchor" id="stats:_frequencies_0"></a>


#### Traits <a class="anchor" id="traits"></a>


In [69]:
df_freq(df, 'Traits')

,count,occurences
Timeless,5,"Creation, Destruction, Ethereal Plane, Tailway, The Infinite Library"
Immeasurable Scope,4,"Dreamworld, Evergrowing Forest, Utopia, Void"
Erratic Time,4,"Dreamworld, Evergrowing Forest, Tartara, Zinctizuctra"
Subjective Gravity,3,"Ethereal Plane, Tailway, Zinctizuctra"
Microgravity,3,"Air Quarter, Dreamworld, Void"
Immeasurable scope,3,"Ethereal Plane, Tartara, The Infinite Library"
Enhanced Magic (Occult),3,"Dreamworld, Ethereal Plane, Void"
Sentient Morphic,3,"Creation, Destruction, Evergrowing Forest"
Negative Essence,2,"Destruction, Shadow Plane"
Flowing Time,2,"Fortress of the Just, Utopia"


#### Category <a class="anchor" id="category_0"></a>


In [70]:
df_freq(df, 'Category')

,count,occurences
Elemental Ring,6,"Air Quarter, Earth Quarter, Elemental Ring, Fire Quarter, Kelzadam, Water Quarter"
Outer,6,"Creation, Destruction, Fortress of the Just, Golden Palace, Hell, Zinctizuctra"
Fifth Dimension,6,"Dreamworld, Ethereal Plane, Evergrowing Forest, Shadow Plane, Utopia, Void"
River Styx,4,"Abyss, Tailway, Tartara, Zaramel"
Elemental,1,Material Plane
Demiplane,1,The Infinite Library


#### Divinities <a class="anchor" id="divinities"></a>


In [71]:
df_freq(df, 'Divinities')

,count,occurences
Celestial Emissaries,3,"Fortress of the Just, Golden Palace, Zaramel"
Fiendish Lords,3,"Abyss, Hell, Tartara"
Yerastu,2,"Fortress of the Just, Tailway"
Fyrlean,2,"Elemental Ring, Fire Quarter"
Hansa,2,"The Infinite Library, Utopia"
Laerion,2,"Fortress of the Just, Golden Palace"
Nentha,2,"Fortress of the Just, Material Plane"
Anhalt,2,"Fortress of the Just, Kelzadam"
Klemnestron,2,"Fire Quarter, Tartara"
Aspects,2,"Evergrowing Forest, Material Plane"


#### Native Inhabitants <a class="anchor" id="native_inhabitants"></a>


In [72]:
df_freq(df, 'Native Inhabitants')

,count,occurences
devoted souls,15,"Creation, Destruction, Evergrowing Forest, Fire Quarter, Fortress of the Just, Golden Palace, Hell, Kelzadam, Shadow Plane, Tailway, Utopia, Void, Water Quarter, Zaramel, Zinctizuctra"
occult dragons,2,"Dreamworld, Ethereal Plane"
tortured souls,2,"Abyss, Tartara"
Elementals,2,"Elemental Ring, Kelzadam"
metallic dragons,2,"Fortress of the Just, Golden Palace"
Angels,2,"Fortress of the Just, Zaramel"
agathions,2,"Evergrowing Forest, Golden Palace"
abominations,2,"Dreamworld, Void"
Demons,1,Abyss
vulga,1,Tartara
